# Download Pretrained Model



In [1]:
import os
from os.path import join
# ROOT_DIR = os.path.abspath("STEGOTest/")
srcDIR = "/home/robotics/Documents/ARCJupyter/STEGOTest/src/"
os.chdir(srcDIR)
saved_models_dir = join("..", "saved_models")
os.makedirs(saved_models_dir, exist_ok=True)

In [2]:
import wget
saved_model_url_root = "https://marhamilresearch4.blob.core.windows.net/stego-public/saved_models/"
saved_model_name = "cocostuff27_vit_base_5.ckpt"
if not os.path.exists(join(saved_models_dir, saved_model_name)):
    wget.download(saved_model_url_root + saved_model_name, join(saved_models_dir, saved_model_name))

# Load pretrained STEGO

In [3]:
from train_segmentation import LitUnsupervisedSegmenter

# This uses DINO weights since it won't find any other ones
model = LitUnsupervisedSegmenter.load_from_checkpoint(join(saved_models_dir, saved_model_name)).cuda()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/robotics/Documents/ARCJupyter/STEGOTest/src/train_segmentation.py:387: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="configs", config_name="train_config.yml")
/home/robotics/anaconda3/envs/pysource/lib/python3.8/site-packages/pytorch_lightning/utilities/migration/migration.py:201: PossibleUserWarning: You have multiple `ModelCheckpoint` callback states in this checkpoint, but we found state keys that would end up colliding with each other after an upgrade, which means we can't differentiate which of your checkpoint callbacks needs which states. At least one of your `ModelCheckpoint` callbacks will not be able to reload the state.
  rank_zero_warn(
Lightning automat

Since no pretrained weights have been provided, we load the reference pretrained DINO weights.


# Imports

In [4]:
from PIL import Image
import requests
from io import BytesIO
from torchvision.transforms.functional import to_tensor
from utils import get_transform
import cv2
import torch.nn.functional as F
from crf import dense_crf
import torch
import time
import numpy as np

# # Visualize with plots
# import matplotlib.pyplot as plt
# from utils import unnorm, remove_axes
# %matplotlib inline

# Query model and pass result through CRF

In [5]:
# Pre-recorded video
vid = cv2.VideoCapture('/home/robotics/Documents/ARCJupyter/STEGOTest/src/videos/ARC_Video.MOV')

# Resolution of vid
# size = (
#     int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
#     int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
# )

# Resolution has to be 448 x 448 since that's what STEGO's numpy shape is
# Maybe try to change this or enlargen the final shape to 1920 x 1080
size = (448, 448)

vidDIR = '/home/robotics/Documents/ARCJupyter/STEGOTest/src/videos/processed/'

# Set properties of output video/image to be read in
codec = cv2.VideoWriter_fourcc(*'DIVX')
output = cv2.VideoWriter(os.path.join(vidDIR, 'ARCSTEGO_448x448_30fps.avi'), codec, 30.0, size)

filename = 'tempimage.png'
tempimgDIR = '/home/robotics/Documents/ARCJupyter/STEGOTest/src/videos/tempimages/'
os.chdir(tempimgDIR)

starttime = time.time()

while(True):
    ret, frame = vid.read()
    if ret == False:
        break
        
    # Read in images/video
    
#    # APPROACH #1
# --------------------------------------------------------------------------------------------------
#    # This approach is interesting because its got a blue tint with some edge/object identification
#     img = Image.fromarray(frame, mode="RGB")
#     transform = get_transform(448, False, "center")
#     img = transform(img).unsqueeze(0).cuda()
# --------------------------------------------------------------------------------------------------
    
    # APPROACH #2
# --------------------------------------------------------------------------------------------------
    # Save frame as 'tempimage.png' each time and process that file
    cv2.imwrite(filename, frame)
    img = cv2.imread(os.path.join(tempimgDIR, filename))
    img = Image.fromarray(img, mode="RGB")
    transform = get_transform(448, False, "center")
    img = transform(img).unsqueeze(0).cuda()
# --------------------------------------------------------------------------------------------------
    
    # Process images/video
    with torch.no_grad():
        code1 = model(img)
        code2 = model(img.flip(dims=[3]))
        code  = (code1 + code2.flip(dims=[3])) / 2
        code = F.interpolate(code, img.shape[-2:], mode='bilinear', align_corners=False)
        linear_probs = torch.log_softmax(model.linear_probe(code), dim=1).cpu()
        cluster_probs = model.cluster_probe(code, 2, log_probs=True).cpu()

        single_img = img[0].cpu()
        linear_pred = dense_crf(single_img, linear_probs[0]).argmax(0)
        cluster_pred = dense_crf(single_img, cluster_probs[0]).argmax(0)
    
        linearres = model.label_cmap[linear_pred].astype('uint8')
        clusterres = model.label_cmap[cluster_pred].astype('uint8')
        
        # Save processed images/video
        output.write(linearres)
        #cv2.imshow('Cluster Predictions', clusterres)
        cv2.imshow('Linear Predictions', linearres)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

vid.release()
output.release()
cv2.destroyAllWindows()

endtime = time.time()
totaltime = endtime - starttime
print('Total Time Taken:', totaltime / 60, 'min(s)')

Total Time Taken: 8.635258456071218 min(s)
